In [1]:
!pip install google-genai==1.3.0

In [2]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [3]:
# @title Gen AI SDK 기본 사용법

from google import genai
client = genai.Client(api_key=GOOGLE_API_KEY)
response = client.models.generate_content(
    model='gemini-2.0-flash', contents='인공지능에 대해 20 단어 이내로 설명하세요.'
)
print(response.text)

인공지능은 인간 지능을 모방하여 학습, 문제 해결, 의사 결정을 수행하는 컴퓨터 시스템입니다.



In [4]:
from google.genai import types
config=types.GenerateContentConfig(
    system_instruction='당신은 한국어로 대화하는 AI 어시스턴트입니다.',
    #temperature= "0.0", #문자열 "0.0"은 float 0.0으로 자동 형 변환됨
    temperature= "zero",
)
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='자기 소개 부탁드립니다.',
    config=config,
)


ValidationError: 1 validation error for GenerateContentConfig
temperature
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='zero', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [5]:
# @title Pydantic 기반의 구조화된 출력

from google.genai import types
from pydantic import BaseModel, Field
from typing import Literal
from pprint import pprint

class Product(BaseModel):
    상품명: str = Field(description="아웃도어 스토어에서 판매하는 의류 이름입니다.")
    사이즈: Literal["S", "M", "L", "XL"]
    가격: int = Field(description="상품의 판매 가격입니다.")

json_schema = Product.model_json_schema()
print("JSON 스키마:")
pprint(json_schema)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents='덩치가 크고 등산을 좋아하는 남성의 옷을 추천해주세요.',
    config=types.GenerateContentConfig(
        response_mime_type='application/json',
        temperature=0.7,  # 0과 1 사이의 값으로 조정
        response_schema=Product
    ),
)
print("\n모델 응답:")
print(response.text)



JSON 스키마:
{'properties': {'가격': {'description': '상품의 판매 가격입니다.',
                       'title': '가격',
                       'type': 'integer'},
                '사이즈': {'enum': ['S', 'M', 'L', 'XL'],
                        'title': '사이즈',
                        'type': 'string'},
                '상품명': {'description': '아웃도어 스토어에서 판매하는 의류 이름입니다.',
                        'title': '상품명',
                        'type': 'string'}},
 'required': ['상품명', '사이즈', '가격'],
 'title': 'Product',
 'type': 'object'}

모델 응답:
{
  "상품명": "남성용 헤비 듀티 등산 자켓",
  "사이즈": "XL",
  "가격": 249000
}
